In [2]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK

__Part 1.__

The Bavarian Motor Company (BMC) manufactures expensive luxury cars in Hamburg, Germany, and exports cars to sell in the United States. The exported cars are shipped from Hamburg to ports in Newark, New Jersey and Jacksonville, Florida. From these ports, the cars are transported by rail or truck to distributors located in Boston, Massachusetts; Columbus, Ohio; Atlanta, Georgia; Richmond, Virginia; and Mobile, Alabama. The below figure shows the possible shipping routes available to the company along with the transportation cost for shipping each car along the indicated path. Currently, 200 cars are available at the port in Newark and 300 are available in Jacksonville. The numbers of cars needed by the distributors in Boston, Columbus, Atlanta, Richmond, and Mobile are 100, 60, 170, 80, and 70, respectively. BMC wants to determine the least costly way of transporting cars from the ports in Newark and Jacksonville to the cities where they are needed.  

1. Formulate the LP and solve it using software of your choice. 
2. Conduct sensitivity analysis and interpret.  
3. Provide the duals and demonstrate strong duality (i.e., complementary slackness).

Strong duality, asserts that the minimal cost in the dual equals the maximal profit in the primal. (https://web.mit.edu/15.053/www/AMP-Chapter-04.pdf)

In [6]:
# Problem (Wine Problem)
# define variables
A1 = LpVariable("A1", 0, None)
A2 = LpVariable("A2", 0, None)
A3 = LpVariable("A3", 0, None)
A4 = LpVariable("A4", 0, None)
B1 = LpVariable("B1", 0, None)
B2 = LpVariable("B2", 0, None)
B3 = LpVariable("B3", 0, None)
B4 = LpVariable("B4", 0, None)

# defines the problem
prob4 = LpProblem("problem", LpMaximize)

# define constraints
prob4 += A1+A2+A3+A4 <= 3500
prob4 += B1+B2+B3+B4 <= 3100
prob4 += A1+B1 <= 1800
prob4 += A2+B2 <= 2300
prob4 += A3+B3 <= 1250
prob4 += A4+B4 <= 1750


# define objective function
prob4 += 39*A1+36*A2+34*A3+32*A4+32*B1+36*B2+37*B3+29*B4

# solve the problem
prob4.writeLP("prob4.lp")
prob4.solve(GLPK(msg=0, options=['--ranges', 'prob4.sen']))
print ("Status:", LpStatus[prob4.status])

# Note, we are only able to get sensitivity information because we are solving
# as a linear program.  If we solved as an Integer Program, then no 
# sensitivity information would be available.
for v in prob4.variables():
    print(v.name, "=", v.varValue)
print ("Objective", value(prob4.objective))
print ("")

f = open("prob4.sen", "r")
print(f.read())

Status: Optimal
A1 = 1800.0
A2 = 450.0
A3 = 0.0
A4 = 1250.0
B1 = 0.0
B2 = 1850.0
B3 = 1250.0
B4 = 0.0
Objective 239250.0

GLPK 5.0  - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  OBJ = 239250 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 _C1          NU    3500.00000        .               -Inf     2250.00000     -32.00000  199250.00000 A4
                                          32.00000    3500.00000     4000.00000          +Inf  255250.00000 _C6

     2 _C2          NU    3100.00000        .               -Inf     1850.00000     -32.00000  199250.00000 A4
                 